<a href="https://colab.research.google.com/github/kiaerii/lab.subbota/blob/main/predlab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import os
os.remove('library.db')

import sqlite3
from datetime import date, datetime, timedelta
from typing import List, Dict, Optional, Any


class LibraryManager:
    def __init__(self, db_path: str = 'library.db'):
        self.db_path = db_path
        self._init_db()

    def _get_connection(self) -> sqlite3.Connection:
        """Создать подключение к базе данных"""
        conn = sqlite3.connect(self.db_path)
        conn.row_factory = sqlite3.Row
        return conn

    def _init_db(self):
        """Инициализация базы данных и таблиц"""
        with self._get_connection() as conn:
            cursor = conn.cursor()

            # Создание таблицы книг
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS books (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    title TEXT NOT NULL,
                    author TEXT NOT NULL,
                    year INTEGER,
                    genre TEXT,
                    is_available BOOLEAN DEFAULT 1
                )
            ''')

            # Создание таблицы читателей
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS readers (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    name TEXT NOT NULL,
                    email TEXT UNIQUE,
                    phone TEXT,
                    registration_date DATE DEFAULT CURRENT_DATE
                )
            ''')

            # Создание таблицы выдачи книг
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS borrowings (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    book_id INTEGER NOT NULL,
                    reader_id INTEGER NOT NULL,
                    borrow_date DATE DEFAULT CURRENT_DATE,
                    return_date DATE,
                    FOREIGN KEY (book_id) REFERENCES books(id),
                    FOREIGN KEY (reader_id) REFERENCES readers(id)
                )
            ''')

            conn.commit()

    def add_book(self, title: str, author: str, year: Optional[int] = None,
                 genre: Optional[str] = None) -> int:
        """Добавить новую книгу в библиотеку"""
        try:
            with self._get_connection() as conn:
                cursor = conn.cursor()
                cursor.execute(
                    '''
                    INSERT INTO books (title, author, year, genre)
                    VALUES (?, ?, ?, ?)
                    ''',
                    (title, author, year, genre)
                )
                conn.commit()
                book_id = cursor.lastrowid
                print(f"Книга '{title}' успешно добавлена (ID: {book_id})")
                return book_id
        except sqlite3.Error as e:
            print(f"Ошибка при добавлении книги: {e}")
            raise

    def add_reader(self, name: str, email: Optional[str] = None,
                   phone: Optional[str] = None) -> int:
        """Зарегистрировать нового читателя"""
        try:
            with self._get_connection() as conn:
                cursor = conn.cursor()
                cursor.execute(
                    '''
                    INSERT INTO readers (name, email, phone)
                    VALUES (?, ?, ?)
                    ''',
                    (name, email, phone)
                )
                conn.commit()
                reader_id = cursor.lastrowid
                print(f"Читатель '{name}' успешно зарегистрирован (ID: {reader_id})")
                return reader_id
        except sqlite3.IntegrityError:
            raise ValueError(f"Читатель с email '{email}' уже зарегистрирован")
        except sqlite3.Error as e:
            print(f"Ошибка при регистрации читателя: {e}")
            raise

    def borrow_book(self, book_id: int, reader_id: int) -> int:
        """
        Выдать книгу читателю
        - Проверить, что книга доступна
        - Обновить статус книги
        - Создать запись о выдаче
        - Использовать транзакцию
        """
        try:
            with self._get_connection() as conn:
                conn.isolation_level = 'EXCLUSIVE'  # Устанавливаем уровень изоляции
                cursor = conn.cursor()

                # Начинаем транзакцию
                cursor.execute("BEGIN TRANSACTION")

                try:
                    # Проверяем доступность книги
                    cursor.execute(
                        '''
                        SELECT title, is_available FROM books
                        WHERE id = ?
                        ''',
                        (book_id,)
                    )
                    book_data = cursor.fetchone()

                    if not book_data:
                        raise ValueError(f"Книга с ID {book_id} не найдена")

                    title = book_data['title']
                    is_available = book_data['is_available']

                    if not is_available:
                        raise ValueError(f"Книга '{title}' уже выдана другому читателю")

                    # Проверяем существование читателя
                    cursor.execute(
                        '''
                        SELECT name FROM readers
                        WHERE id = ?
                        ''',
                        (reader_id,)
                    )
                    reader_data = cursor.fetchone()

                    if not reader_data:
                        raise ValueError(f"Читатель с ID {reader_id} не найден")

                    reader_name = reader_data['name']

                    # Обновляем статус книги
                    cursor.execute(
                        '''
                        UPDATE books
                        SET is_available = 0
                        WHERE id = ?
                        ''',
                        (book_id,)
                    )

                    # Создаем запись о выдаче
                    cursor.execute(
                        '''
                        INSERT INTO borrowings (book_id, reader_id)
                        VALUES (?, ?)
                        ''',
                        (book_id, reader_id)
                    )

                    borrowing_id = cursor.lastrowid

                    # Фиксируем транзакцию
                    conn.commit()

                    print(f"Книга '{title}' успешно выдана читателю {reader_name}")
                    return borrowing_id

                except Exception as e:
                    # Откатываем транзакцию при ошибке
                    conn.rollback()
                    raise

        except sqlite3.Error as e:
            print(f"Ошибка базы данных при выдаче книги: {e}")
            raise

    def return_book(self, borrowing_id: int) -> None:
        """
        Вернуть книгу в библиотеку
        - Обновить статус книги
        - Установить дату возврата
        - Использовать транзакцию
        """
        try:
            with self._get_connection() as conn:
                conn.isolation_level = 'EXCLUSIVE'
                cursor = conn.cursor()

                cursor.execute("BEGIN TRANSACTION")

                try:
                    # Получаем информацию о выдаче
                    cursor.execute(
                        '''
                        SELECT b.id as book_id, bk.title, b.reader_id
                        FROM borrowings b
                        JOIN books bk ON b.book_id = bk.id
                        WHERE b.id = ? AND b.return_date IS NULL
                        ''',
                        (borrowing_id,)
                    )
                    borrowing_data = cursor.fetchone()

                    if not borrowing_data:
                        raise ValueError(f"Выдача с ID {borrowing_id} не найдена или книга уже возвращена")

                    book_id = borrowing_data['book_id']
                    title = borrowing_data['title']

                    # Обновляем статус книги
                    cursor.execute(
                        '''
                        UPDATE books
                        SET is_available = 1
                        WHERE id = ?
                        ''',
                        (book_id,)
                    )

                    # Устанавливаем дату возврата
                    cursor.execute(
                        '''
                        UPDATE borrowings
                        SET return_date = CURRENT_DATE
                        WHERE id = ?
                        ''',
                        (borrowing_id,)
                    )

                    conn.commit()
                    print(f"Книга '{title}' возвращена в библиотеку")

                except Exception as e:
                    conn.rollback()
                    raise

        except sqlite3.Error as e:
            print(f"Ошибка базы данных при возврате книги: {e}")
            raise

    def find_available_books(self, author: Optional[str] = None,
                            genre: Optional[str] = None) -> List[Dict[str, Any]]:
        """Найти доступные книги (с фильтрацией по автору/жанру)"""
        try:
            with self._get_connection() as conn:
                cursor = conn.cursor()

                query = '''
                    SELECT id, title, author, year, genre
                    FROM books
                    WHERE is_available = 1
                '''
                params = []

                if author:
                    query += ' AND author LIKE ?'
                    params.append(f'%{author}%')

                if genre:
                    query += ' AND genre LIKE ?'
                    params.append(f'%{genre}%')

                query += ' ORDER BY title'

                cursor.execute(query, params)
                rows = cursor.fetchall()

                # Преобразуем результат в список словарей
                result = []
                for row in rows:
                    result.append({
                        'id': row['id'],
                        'title': row['title'],
                        'author': row['author'],
                        'year': row['year'],
                        'genre': row['genre']
                    })

                return result

        except sqlite3.Error as e:
            print(f"Ошибка при поиске книг: {e}")
            return []

    def get_reader_borrowings(self, reader_id: int) -> List[Dict[str, Any]]:
        """Получить список текущих выдач читателя"""
        try:
            with self._get_connection() as conn:
                cursor = conn.cursor()

                cursor.execute(
                    '''
                    SELECT b.id as borrowing_id, bk.title, bk.author,
                           b.borrow_date, b.return_date
                    FROM borrowings b
                    JOIN books bk ON b.book_id = bk.id
                    WHERE b.reader_id = ?
                    ORDER BY b.borrow_date DESC
                    ''',
                    (reader_id,)
                )

                rows = cursor.fetchall()

                result = []
                for row in rows:
                    result.append({
                        'borrowing_id': row['borrowing_id'],
                        'title': row['title'],
                        'author': row['author'],
                        'borrow_date': row['borrow_date'],
                        'return_date': row['return_date'],
                        'is_returned': row['return_date'] is not None
                    })

                return result

        except sqlite3.Error as e:
            print(f"Ошибка при получении списка выдач: {e}")
            return []

    def get_overdue_borrowings(self, days: int = 30) -> List[Dict[str, Any]]:
        """Найти просроченные выдачи (больше N дней)"""
        try:
            with self._get_connection() as conn:
                cursor = conn.cursor()

                # Вычисляем дату, до которой нужно было вернуть книгу
                overdue_date = (date.today() - timedelta(days=days)).isoformat()

                cursor.execute(
                    '''
                    SELECT b.id as borrowing_id, bk.title, bk.author,
                           r.name as reader_name, r.email, r.phone,
                           b.borrow_date,
                           julianday(CURRENT_DATE) - julianday(b.borrow_date) as days_borrowed
                    FROM borrowings b
                    JOIN books bk ON b.book_id = bk.id
                    JOIN readers r ON b.reader_id = r.id
                    WHERE b.return_date IS NULL
                    AND b.borrow_date <= ?
                    ORDER BY b.borrow_date ASC
                    ''',
                    (overdue_date,)
                )

                rows = cursor.fetchall()

                result = []
                for row in rows:
                    result.append({
                        'borrowing_id': row['borrowing_id'],
                        'title': row['title'],
                        'author': row['author'],
                        'reader_name': row['reader_name'],
                        'email': row['email'],
                        'phone': row['phone'],
                        'borrow_date': row['borrow_date'],
                        'days_borrowed': int(row['days_borrowed'])
                    })

                return result

        except sqlite3.Error as e:
            print(f"Ошибка при поиске просроченных выдач: {e}")
            return []

    def get_book_info(self, book_id: int) -> Optional[Dict[str, Any]]:
        """Получить полную информацию о книге"""
        try:
            with self._get_connection() as conn:
                cursor = conn.cursor()

                cursor.execute(
                    '''
                    SELECT b.*,
                           br.borrow_date, br.return_date,
                           r.name as reader_name
                    FROM books b
                    LEFT JOIN borrowings br ON b.id = br.book_id AND br.return_date IS NULL
                    LEFT JOIN readers r ON br.reader_id = r.id
                    WHERE b.id = ?
                    ''',
                    (book_id,)
                )

                row = cursor.fetchone()
                if row:
                    return {
                        'id': row['id'],
                        'title': row['title'],
                        'author': row['author'],
                        'year': row['year'],
                        'genre': row['genre'],
                        'is_available': bool(row['is_available']),
                        'current_reader': row['reader_name'],
                        'borrow_date': row['borrow_date']
                    }
                return None

        except sqlite3.Error as e:
            print(f"Ошибка при получении информации о книге: {e}")
            return None


def main():
    """Пример использования библиотечной системы"""
    library = LibraryManager()


    # Добавляем книги
    print("\n1. Добавление книг:")
    library.add_book("Преступление и наказание", "Федор Достоевский", 1866, "Роман")
    library.add_book("Мастер и Маргарита", "Михаил Булгаков", 1967, "Роман")
    library.add_book("1984", "Джордж Оруэлл", 1949, "Антиутопия")
    library.add_book("Скотный двор", "Джордж Оруэлл", 1945, "Сатира")

    # Регистрируем читателей
    print("\n2. Регистрация читателей:")
    library.add_reader("Иван Иванов", "ivan@mail.com", "+79161234567")
    library.add_reader("Петр Петров", "petr@mail.com", "+79167654321")

    # Выдаем книги
    print("\n3. Выдача книг:")
    library.borrow_book(1, 1)  # Иван берет "Преступление и наказание"
    library.borrow_book(2, 2)  # Петр берет "Мастер и Маргарита"

    # Пытаемся выдать уже занятую книгу
    print("\n4. Попытка выдать уже занятую книгу:")
    try:
        library.borrow_book(1, 2)  # Должна быть ошибка
    except Exception as e:
        print(f"Ошибка: {e}")

    # Ищем доступные книги
    print("\n5. Поиск доступных книг:")
    available = library.find_available_books(author="Джордж Оруэлл")
    print("Доступные книги Оруэлла:")
    for book in available:
        print(f"  - {book['title']} ({book['year']}, {book['genre']})")

    # Проверяем информацию о книге
    print("\n6. Информация о выданной книге:")
    book_info = library.get_book_info(1)
    if book_info:
        print(f"Книга: {book_info['title']}")
        print(f"Статус: {'Доступна' if book_info['is_available'] else 'Выдана'}")
        if not book_info['is_available']:
            print(f"У читателя: {book_info['current_reader']}")

    # Возвращаем книгу
    print("\n7. Возврат книги:")
    library.return_book(1)  # Возвращаем первую выдачу

    # Проверяем, что книга снова доступна
    print("\n8. Все доступные книги после возврата:")
    available = library.find_available_books()
    for book in available:
        print(f"  - {book['title']} - {book['author']}")

    # Проверяем выдачи читателя
    print("\n9. Выдачи читателя Иван Иванов:")
    borrowings = library.get_reader_borrowings(1)
    for borrowing in borrowings:
        status = "возвращена" if borrowing['is_returned'] else "на руках"
        print(f"  - {borrowing['title']}: {status}")

if __name__ == "__main__":
    main()


1. Добавление книг:
Книга 'Преступление и наказание' успешно добавлена (ID: 1)
Книга 'Мастер и Маргарита' успешно добавлена (ID: 2)
Книга '1984' успешно добавлена (ID: 3)
Книга 'Скотный двор' успешно добавлена (ID: 4)

2. Регистрация читателей:
Читатель 'Иван Иванов' успешно зарегистрирован (ID: 1)
Читатель 'Петр Петров' успешно зарегистрирован (ID: 2)

3. Выдача книг:
Книга 'Преступление и наказание' успешно выдана читателю Иван Иванов
Книга 'Мастер и Маргарита' успешно выдана читателю Петр Петров

4. Попытка выдать уже занятую книгу:
Ошибка: Книга 'Преступление и наказание' уже выдана другому читателю

5. Поиск доступных книг:
Доступные книги Оруэлла:
  - 1984 (1949, Антиутопия)
  - Скотный двор (1945, Сатира)

6. Информация о выданной книге:
Книга: Преступление и наказание
Статус: Выдана
У читателя: Иван Иванов

7. Возврат книги:
Книга 'Преступление и наказание' возвращена в библиотеку

8. Все доступные книги после возврата:
  - 1984 - Джордж Оруэлл
  - Преступление и наказание - Ф